# **Appendix: Non Linear Regression Project (Corrected for t_orig)**
**Author:** Mainak Sarkar | 230619

## Loading the Data Set

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.stats import kstest, norm
from scipy.linalg import pinv
import warnings
warnings.filterwarnings('ignore')

# Loading data
data = pd.read_csv('set-49.dat', sep='\s+', header=None)
print(data.head())

# Extract time and observed values
t_orig = np.array(data[0], dtype=float)  # Original time (1-60)
y1 = np.array(data[1], dtype=float)  # Observed y(t) values
n = len(y1)  # Get n from y1

# Model 1: $y(t) = \\alpha_0 + \\alpha_1 e^{\\beta_1 t} + \\alpha_2 e^{\\beta_2 t} + \\epsilon(t)$

## 1.1 Fitting

Method changed to Non-Linear Least Squares (NLS) using scipy.optimize for stability with original t values

In [ ]:
# --- NLS implementation for Model 1 ---

# Use original time values
tt_m1 = t_orig
yy_m1 = y1

# Model 1 function: p = [a0, a1, b1, a2, b2]
def f_model_1(p, t):
    a0 = p[0]
    a1 = p[1]
    b1 = p[2]
    a2 = p[3]
    b2 = p[4]
    return a0 + a1 * np.exp(b1 * t) + a2 * np.exp(b2 * t)

# SSE cost function for Model 1
def sse_1(p):
    yhat = f_model_1(p, tt_m1)
    if not np.all(np.isfinite(yhat)):
        return 1e10  # Penalize explosions
    return np.sum((yy_m1 - yhat)**2)

# Initial guesses
init_1 = np.array([1.0, 0.2, 0.001, 2.0, -0.3])

# Run optimization
fit_1 = minimize(sse_1, init_1, method='L-BFGS-B', options={'maxiter': 2000})
params_1 = fit_1.x

parameter_estimates_1 = params_1
alpha0_1 = params_1[0]
alpha1_1 = params_1[1]
beta1_1 = params_1[2]
alpha2_1 = params_1[3]
beta2_1 = params_1[4]

fitted_values_m1 = f_model_1(params_1, tt_m1)
residuals_m1 = yy_m1 - fitted_values_m1

## 1.2 Checking the Model Accuracy for Model 1

In [ ]:
# Calculate model accuracy metrics
rmse_1 = np.sqrt(np.mean(residuals_m1**2))
rss_1 = np.sum(residuals_m1**2)
tss_1 = np.sum((y1 - np.mean(y1))**2)
r_squared_1 = 1 - (rss_1 / tss_1)

k_1 = 5  # Number of parameters (alpha0, alpha1, beta1, alpha2, beta2)
adjusted_r_squared_1 = 1 - ((1 - r_squared_1) * (n - 1) / (n - k_1 - 1))

print(f"RMSE: {rmse_1}")
print(f"RSS: {rss_1}")
print(f"TSS: {tss_1}")
print(f"R-squared: {r_squared_1}")
print(f"Adjusted R-squared: {adjusted_r_squared_1}")

## 1.3 Calculating the Jacobian Matrix, FIM and CI

In [ ]:
# Define the Jacobian of the model (partial derivatives)
def jacobian_1(t, alpha0, alpha1, beta1, alpha2, beta2):
    d_alpha0 = 1
    d_alpha1 = np.exp(beta1 * t)
    d_beta1 = alpha1 * t * np.exp(beta1 * t)
    d_alpha2 = np.exp(beta2 * t)
    d_beta2 = alpha2 * t * np.exp(beta2 * t)
    return np.array([d_alpha0, d_alpha1, d_beta1, d_alpha2, d_beta2])

# Fisher Information matrix calculation
def fisher_information(t_vec, params, jac_func, sigma2):
    n_obs = len(t_vec)
    n_params = len(params)
    J = np.zeros((n_obs, n_params))
    
    for i in range(n_obs):
        J[i, :] = jac_func(t_vec[i], params[0], params[1], params[2], params[3], params[4])
    
    # FIM is (1/sigma2) * (J'J)
    fisher_matrix = (1 / (n_obs * sigma2)) * (J.T @ J)
    
    # Regularization (if required)
    lambda_reg = 1e-6
    fisher_matrix_regularized = fisher_matrix + lambda_reg * np.eye(n_params)
    
    return fisher_matrix_regularized

# Confidence Intervals calculation
def confidence_intervals(fisher_matrix, parameter_estimates, alpha=0.05):
    # Inverse Fisher Information Matrix (for variance)
    fisher_inv = pinv(fisher_matrix)
    
    # Standard errors
    se = np.sqrt(np.diag(fisher_inv))
    
    # Z value for 95% confidence
    z_value = norm.ppf(1 - alpha / 2)
    
    # Confidence intervals
    ci_lower = parameter_estimates - z_value * se
    ci_upper = parameter_estimates + z_value * se
    
    return {'lower': ci_lower, 'upper': ci_upper, 'se': se}

sigma2_1 = rss_1 / (n - k_1)

# Calculate Fisher Information Matrix for the model parameters
fisher_matrix_1 = fisher_information(tt_m1, parameter_estimates_1, jacobian_1, sigma2_1)

# Calculate Confidence Intervals
ci_1 = confidence_intervals(fisher_matrix_1, parameter_estimates_1)

# Print the results
print("--- Model 1 Results ---")
print("Parameters (alpha0, alpha1, beta1, alpha2, beta2):")
print(parameter_estimates_1)
print("\nFisher Information Matrix:")
print(fisher_matrix_1)
print("\nConfidence Intervals:")
print(f"Lower: {ci_1['lower']}")
print(f"Upper: {ci_1['upper']}")
print(f"Standard Errors: {ci_1['se']}")

## 1.4 Plots

In [ ]:
# Set up the plotting area with 2x2 subplots
fig, axes = plt.subplots(2, 2, figsize=(10, 8))

# Residuals plot
axes[0, 0].scatter(t_orig, residuals_m1, color='black', s=30, alpha=0.7)
axes[0, 0].axhline(y=0, color='red', linestyle='-')
axes[0, 0].set_title('1.6: Residuals vs. Time (Model 1)')
axes[0, 0].set_xlabel('Time (Original)')
axes[0, 0].set_ylabel('Residuals')
axes[0, 0].grid(True, alpha=0.3)

# Q-Q plot of residuals
from scipy import stats
stats.probplot(residuals_m1, dist="norm", plot=axes[0, 1])
axes[0, 1].set_title('1.7(a): Q-Q Plot of Residuals (Model 1)')
axes[0, 1].grid(True, alpha=0.3)

# Histogram of residuals
axes[1, 0].hist(residuals_m1, bins=10, color='lightblue', edgecolor='black')
axes[1, 0].set_title('1.7(b): Histogram of Residuals (Model 1)')
axes[1, 0].set_xlabel('Residuals')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].grid(True, alpha=0.3)

# Fitted values plot
axes[1, 1].plot(t_orig, y1, color='blue', linewidth=2, label='Observed Data')
axes[1, 1].plot(t_orig, fitted_values_m1, color='red', linewidth=2, label='Model-1 Fit')
axes[1, 1].set_title('1.8: Observed vs. Fitted Data (Model 1)')
axes[1, 1].set_xlabel('Time (Original)')
axes[1, 1].set_ylabel('Values')
axes[1, 1].legend(loc='upper right')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 1.5 Applying KS Test

In [ ]:
# Standardize residuals
standardized_residuals_1 = (residuals_m1 - np.mean(residuals_m1)) / np.std(residuals_m1)

# Perform the Kolmogorov-Smirnov test
ks_test_1 = kstest(standardized_residuals_1, 'norm', args=(0, 1))

# Display KS test results
print("--- Model 1 KS Test ---")
print(f"KS Test Statistic: {ks_test_1.statistic}")
print(f"KS Test p-value: {ks_test_1.pvalue}")

---
# Model 2: $y(t) = \\frac{\\beta_0 + \\beta_1 t}{\\alpha_0 + \\alpha_1 t} + \\epsilon(t)$

No changes needed, this model already uses `t_orig`

## 2.1 Fitting

In [ ]:
# Load and align t and y again
tt = t_orig
yy = y1
assert len(tt) == len(yy), "Length mismatch between tt and yy"

# Define model [a0, a1, b0, b1]
def f_model_2(p, t):
    a0 = p[0]
    a1 = p[1]
    b0 = p[2]
    b1 = p[3]
    denom = b0 + b1 * t
    
    # Avoid division by zero
    if np.any(np.abs(denom) < 1e-8):
        return np.full_like(t, np.inf, dtype=float)
    
    return (a0 + a1 * t) / denom

# Define SSE cost function
def sse_2(p):
    yhat = f_model_2(p, tt)
    if not np.all(np.isfinite(yhat)):
        return 1e10  # Penalize bad parameters
    return np.sum((yy - yhat)**2)

# Initial guess from simple linear fit
fit_poly_lin = np.polyfit(tt, yy, 1)
a0_init = fit_poly_lin[1]  # Intercept
a1_init = fit_poly_lin[0]  # Slope
b0_init = 1.0  # Guess
b1_init = 0.1  # Guess
init_2 = np.array([a0_init, a1_init, b0_init, b1_init])

# Use L-BFGS-B with bounds
fit_2 = minimize(sse_2, init_2, method='L-BFGS-B',
                bounds=[(-10, 10), (-10, 10), (0.01, 20), (-10, 20)],
                options={'maxiter': 2000})

params_2 = fit_2.x

a0_2 = params_2[0]
a1_2 = params_2[1]
b0_2 = params_2[2]
b1_2 = params_2[3]

# Final predictions and residuals
fitted_values_2 = f_model_2(params_2, tt)
residuals_2 = yy - fitted_values_2

## 2.2 Checking the Model Accuracy for Model 2

In [ ]:
# Calculate model accuracy metrics
rmse_2 = np.sqrt(np.mean(residuals_2**2))
rss_2 = np.sum(residuals_2**2)
tss_2 = np.sum((yy - np.mean(yy))**2)
r_squared_2 = 1 - (rss_2 / tss_2)

k_2 = 4  # Number of parameters (a0, a1, b0, b1)
adjusted_r_squared_2 = 1 - ((1 - r_squared_2) * (n - 1) / (n - k_2 - 1))

print(f"RMSE: {rmse_2}")
print(f"RSS: {rss_2}")
print(f"TSS: {tss_2}")
print(f"R-squared: {r_squared_2}")
print(f"Adjusted R-squared: {adjusted_r_squared_2}")

## 2.3 Calculating the Jacobian Matrix, FIM and CI

In [ ]:
# Define the Jacobian for Model 2
def jacobian_2(t, a0, a1, b0, b1):
    denom = b0 + b1 * t
    d_a0 = 1 / denom
    d_a1 = t / denom
    d_b0 = -(a0 + a1 * t) / (denom**2)
    d_b1 = -t * (a0 + a1 * t) / (denom**2)
    return np.array([d_a0, d_a1, d_b0, d_b1])

# Fisher Information matrix calculation (4-param version)
def fisher_information_4param(t_vec, params, jac_func, sigma2):
    n_obs = len(t_vec)
    n_params = len(params)
    J = np.zeros((n_obs, n_params))
    
    for i in range(n_obs):
        J[i, :] = jac_func(t_vec[i], params[0], params[1], params[2], params[3])
    
    fisher_matrix = (1 / (n_obs * sigma2)) * (J.T @ J)
    
    lambda_reg = 1e-6
    fisher_matrix_regularized = fisher_matrix + lambda_reg * np.eye(n_params)
    
    return fisher_matrix_regularized

# Confidence Intervals calculation (4-param version)
def confidence_intervals_4param(fisher_matrix, parameter_estimates, alpha=0.05):
    fisher_inv = pinv(fisher_matrix)
    se = np.sqrt(np.diag(fisher_inv))
    z_value = norm.ppf(1 - alpha / 2)
    ci_lower = parameter_estimates - z_value * se
    ci_upper = parameter_estimates + z_value * se
    return {'lower': ci_lower, 'upper': ci_upper, 'se': se}

# Calculate sigma^2 estimate
sigma2_2 = rss_2 / (n - k_2)

# Calculate Fisher Information Matrix
fisher_matrix_2 = fisher_information_4param(tt, params_2, jacobian_2, sigma2_2)

# Calculate Confidence Intervals
ci_2 = confidence_intervals_4param(fisher_matrix_2, params_2)

# Print the results
print("--- Model 2 Results ---")
print("Parameters (a0, a1, b0, b1):")
print(params_2)
print("\nFisher Information Matrix:")
print(fisher_matrix_2)
print("\nConfidence Intervals:")
print(f"Lower: {ci_2['lower']}")
print(f"Upper: {ci_2['upper']}")
print(f"Standard Errors: {ci_2['se']}")

## 2.4 Plots

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 8))

# Residuals plot
axes[0, 0].scatter(tt, residuals_2, color='black', s=30, alpha=0.7)
axes[0, 0].axhline(y=0, color='red', linestyle='-')
axes[0, 0].set_title('2.6: Residuals vs. Time (Model 2)')
axes[0, 0].set_xlabel('Time (Original)')
axes[0, 0].set_ylabel('Residuals')
axes[0, 0].grid(True, alpha=0.3)

# Q-Q plot of residuals
stats.probplot(residuals_2, dist="norm", plot=axes[0, 1])
axes[0, 1].set_title('2.7(a): Q-Q Plot of Residuals (Model 2)')
axes[0, 1].grid(True, alpha=0.3)

# Histogram of residuals
axes[1, 0].hist(residuals_2, bins=10, color='lightblue', edgecolor='black')
axes[1, 0].set_title('2.7(b): Histogram of Residuals (Model 2)')
axes[1, 0].set_xlabel('Residuals')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].grid(True, alpha=0.3)

# Fitted values plot
tt_ord = np.argsort(tt)
axes[1, 1].plot(tt[tt_ord], yy[tt_ord], color='blue', linewidth=2, label='Observed Data')
axes[1, 1].plot(tt[tt_ord], fitted_values_2[tt_ord], color='red', linewidth=2, label='Model-2 Fit')
axes[1, 1].set_title('2.8: Observed vs. Fitted Data (Model 2)')
axes[1, 1].set_xlabel('Time (Original)')
axes[1, 1].set_ylabel('Values')
axes[1, 1].legend(loc='upper right')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2.5 Applying KS Test

In [ ]:
# Standardize residuals
standardized_residuals_2 = (residuals_2 - np.mean(residuals_2)) / np.std(residuals_2)

# Perform the Kolmogorov-Smirnov test
ks_test_2 = kstest(standardized_residuals_2, 'norm', args=(0, 1))

# Display KS test results
print("--- Model 2 KS Test ---")
print(f"KS Test Statistic: {ks_test_2.statistic}")
print(f"KS Test p-value: {ks_test_2.pvalue}")

---
# Model 3: $y(t) = \\beta_0 + \\beta_1 t + \\beta_2 t^2 + \\beta_3 t^3 + \\beta_4 t^4 + \\epsilon(t)$

Updated to use `t_orig` instead of rescaled t

## 3.1 Fitting

In [ ]:
# Fit a 4th-degree polynomial regression model using ORIGINAL t
# Create design matrix
X = np.column_stack([np.ones(len(t_orig)), t_orig, t_orig**2, t_orig**3, t_orig**4])

# Fit using least squares
parameter_estimates_3 = np.linalg.lstsq(X, y1, rcond=None)[0]

# Calculate fitted values and residuals
fitted_values_3 = X @ parameter_estimates_3
residuals_3 = y1 - fitted_values_3

## 3.2 Checking the Model Accuracy for Model 3

In [ ]:
# Calculate model accuracy metrics
rmse_3 = np.sqrt(np.mean(residuals_3**2))
rss_3 = np.sum(residuals_3**2)
tss_3 = np.sum((y1 - np.mean(y1))**2)
r_squared_3 = 1 - (rss_3 / tss_3)

k_3 = 5  # Number of parameters (beta0, beta1, beta2, beta3, beta4)
adjusted_r_squared_3 = 1 - ((1 - r_squared_3) * (n - 1) / (n - k_3 - 1))

print(f"RMSE: {rmse_3}")
print(f"RSS: {rss_3}")
print(f"TSS: {tss_3}")
print(f"R-squared: {r_squared_3}")
print(f"Adjusted R-squared: {adjusted_r_squared_3}")

## 3.3 Calculating the Jacobian Matrix, FIM and CI

In [ ]:
# Define the Jacobian of the model
def jacobian_3(t, beta0, beta1, beta2, beta3, beta4):
    d_beta0 = 1
    d_beta1 = t
    d_beta2 = t**2
    d_beta3 = t**3
    d_beta4 = t**4
    return np.array([d_beta0, d_beta1, d_beta2, d_beta3, d_beta4])

# Fisher Information matrix calculation (5-param version)
def fisher_information_5param(t_vec, params, jac_func, sigma2):
    n_obs = len(t_vec)
    n_params = len(params)
    J = np.zeros((n_obs, n_params))
    
    for i in range(n_obs):
        J[i, :] = jac_func(t_vec[i], params[0], params[1], params[2], params[3], params[4])
    
    fisher_matrix = (1 / (n_obs * sigma2)) * (J.T @ J)
    
    lambda_reg = 1e-6
    fisher_matrix_regularized = fisher_matrix + lambda_reg * np.eye(n_params)
    
    return fisher_matrix_regularized

# Confidence Intervals calculation (5-param version)
def confidence_intervals_5param(fisher_matrix, parameter_estimates, alpha=0.05):
    fisher_inv = pinv(fisher_matrix)
    se = np.sqrt(np.diag(fisher_inv))
    z_value = norm.ppf(1 - alpha / 2)
    ci_lower = parameter_estimates - z_value * se
    ci_upper = parameter_estimates + z_value * se
    return {'lower': ci_lower, 'upper': ci_upper, 'se': se}

beta0 = parameter_estimates_3[0]
beta1 = parameter_estimates_3[1]
beta2 = parameter_estimates_3[2]
beta3 = parameter_estimates_3[3]
beta4 = parameter_estimates_3[4]

# Calculate sigma^2 estimate
sigma2_3 = rss_3 / (n - k_3)

# Calculate Fisher Information Matrix using t_orig
fisher_matrix_3 = fisher_information_5param(t_orig, parameter_estimates_3, jacobian_3, sigma2_3)

# Calculate Confidence Intervals
ci_3 = confidence_intervals_5param(fisher_matrix_3, parameter_estimates_3)

# Print the results
print("--- Model 3 Results ---")
print("Parameters (beta0, beta1, beta2, beta3, beta4):")
print(parameter_estimates_3)
print("\nFisher Information Matrix:")
print(fisher_matrix_3)
print("\nConfidence Intervals:")
print(f"Lower: {ci_3['lower']}")
print(f"Upper: {ci_3['upper']}")
print(f"Standard Errors: {ci_3['se']}")

## 3.4 Plots

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 8))

# Residuals Plot for Model 3
axes[0, 0].scatter(t_orig, residuals_3, color='black', s=30, alpha=0.7)
axes[0, 0].axhline(y=0, color='red', linestyle='-')
axes[0, 0].set_title('3.6: Residuals vs. Time (Model 3)')
axes[0, 0].set_xlabel('Time (Original)')
axes[0, 0].set_ylabel('Residuals')
axes[0, 0].grid(True, alpha=0.3)

# Q-Q Plot for Model 3 Residuals
stats.probplot(residuals_3, dist="norm", plot=axes[0, 1])
axes[0, 1].set_title('3.7(a): Q-Q Plot of Residuals (Model 3)')
axes[0, 1].grid(True, alpha=0.3)

# Histogram of Residuals for Model 3
axes[1, 0].hist(residuals_3, bins=10, color='lightblue', edgecolor='black')
axes[1, 0].set_title('3.7(b): Histogram of Residuals (Model 3)')
axes[1, 0].set_xlabel('Residuals')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].grid(True, alpha=0.3)

# Observed vs. Fitted Data Plot for Model 3
axes[1, 1].plot(t_orig, y1, color='blue', linewidth=2, label='Observed Data')
axes[1, 1].plot(t_orig, fitted_values_3, color='red', linewidth=2, label='Model-3 Fit')
axes[1, 1].set_title('3.8: Observed vs. Fitted Data (Model 3)')
axes[1, 1].set_xlabel('Time (Original)')
axes[1, 1].set_ylabel('Values')
axes[1, 1].legend(loc='upper right')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3.5 Applying KS Test

In [ ]:
# Standardize residuals
standardized_residuals_3 = (residuals_3 - np.mean(residuals_3)) / np.std(residuals_3)

# Perform the Kolmogorov-Smirnov test
ks_test_3 = kstest(standardized_residuals_3, 'norm', args=(0, 1))

# Display KS test results
print("--- Model 3 KS Test ---")
print(f"KS Test Statistic: {ks_test_3.statistic}")
print(f"KS Test p-value: {ks_test_3.pvalue}")

---
# 4.0 Model Comparison

This section provides a summary of all three models to help determine the 'best' fit.

In [ ]:
# Create a comparison table
model_names = ["Model 1 (Exponential)", "Model 2 (Rational [1/1])", "Model 3 (Poly 4th deg)"]
rss_values = [rss_1, rss_2, rss_3]
r_squared_values = [r_squared_1, r_squared_2, r_squared_3]
adj_r_squared_values = [adjusted_r_squared_1, adjusted_r_squared_2, adjusted_r_squared_3]
ks_p_values = [ks_test_1.pvalue, ks_test_2.pvalue, ks_test_3.pvalue]

comparison_df = pd.DataFrame({
    'Model': model_names,
    'RSS': rss_values,
    'R_Squared': r_squared_values,
    'Adjusted_R_Squared': adj_r_squared_values,
    'KS_Test_p_value': ks_p_values
})

print("\n## Model Fit Comparison\n")
print(comparison_df.to_string(index=False))

print("\n### Interpretation Guide\n")
print("* **RSS (Residual Sum of Squares):** Lower is better.")
print("* **Adjusted R-Squared:** Higher is better (adjusts for the number of parameters).")
print("* **KS Test p-value:** A value < 0.05 suggests the residuals are *not* normally distributed, which violates a model assumption.")